# Opening Problem file

In [9]:
import re
from typing import Dict, List, Tuple
import os


def parse_mptsp_problem(file_path: str) -> Dict:
    with open(file_path + '/prob.txt', 'r') as file:
        content = file.read()

    # Initialize the data structure to store parsed information
    problem_data = {
        'name': '',
        'comment': '',
        'type': '',
        'dimension': 0,
        'n_path': 0,
        'edge_weight_type': '',
        'edge_data_format': '',
        'nodes': [],
        'edges': []
    }

    # Parse the header information
    header_pattern = r'NAME\s*:\s*(.+)\nCOMMENT\s*:\s*(.+)\nTYPE\s*:\s*(.+)\nDIMENSION\s*:\s*(\d+)\nN_PATH\s*:\s*(\d+)\nEDGE_WEIGHT_TYPE\s*:\s*(.+)\nEDGE_DATA_FORMAT\s*:\s*(.+)'
    header_match = re.search(header_pattern, content, re.MULTILINE)
    
    if header_match:
        problem_data['name'] = header_match.group(1).strip()
        problem_data['comment'] = header_match.group(2).strip()
        problem_data['type'] = header_match.group(3).strip()
        problem_data['dimension'] = int(header_match.group(4))
        problem_data['n_path'] = int(header_match.group(5))
        problem_data['edge_weight_type'] = header_match.group(6).strip()
        problem_data['edge_data_format'] = header_match.group(7).strip()

    # Parse node coordinates
    node_section = re.search(r'NODE_COORD_SECTION([\s\S]*?)EDGE_WEIGHT_SECTION', content)
    if node_section:
        node_lines = node_section.group(1).strip().split('\n')
        for line in node_lines:
            node_id, x, y = map(float, line.split())
            problem_data['nodes'].append((int(node_id), x, y))

    # Parse edge weights
    edge_section = re.search(r'EDGE_WEIGHT_SECTION([\s\S]*?)EOF', content)
    if edge_section:
        edge_lines = edge_section.group(1).strip().split('\n')
        for line in edge_lines:
            node1, node2, weight = map(int, line.split())
            problem_data['edges'].append((node1, node2, weight))

    return problem_data

# Example usage
if __name__ == "__main__":
    file_path = "/Users/vivekchaudhary/Documents/progressive_hedging/MPTSPs/MPTSPs_D0_50"  # Replace with the actual file path
    parsed_data = parse_mptsp_problem(file_path)
    
    print(f"Problem Name: {parsed_data['name']}")
    print(f"Problem Type: {parsed_data['type']}")
    print(f"Dimension: {parsed_data['dimension']}")
    print(f"Number of Paths: {parsed_data['n_path']}")
    print(f"Number of Nodes: {len(parsed_data['nodes'])}")
    print(f"Number of Edges: {len(parsed_data['edges'])}")
    
    # Print first few nodes and edges as examples
    print("\nFirst 5 Nodes:")
    for node in parsed_data['nodes'][:5]:
        print(f"Node {node[0]}: ({node[1]}, {node[2]})")
    
    print("\nFirst 5 Edges:")
    for edge in parsed_data['edges'][:5]:
        print(f"Edge: {edge[0]} - {edge[1]}, Weight: {edge[2]}")

Problem Name: 
Problem Type: 
Dimension: 0
Number of Paths: 0
Number of Nodes: 54
Number of Edges: 2916

First 5 Nodes:
Node 1: (13972.02909365901, 13982.061694884402)
Node 2: (13437.036638235822, 13792.65423067852)
Node 3: (13672.680326078447, 13968.708877261386)
Node 4: (17202.241249039205, 14696.263504889848)
Node 5: (20451.062770290682, 16272.699684772444)

First 5 Edges:
Edge: 1 - 1, Weight: 0
Edge: 1 - 2, Weight: 312
Edge: 1 - 3, Weight: 141
Edge: 1 - 4, Weight: 1699
Edge: 1 - 5, Weight: 3966


# Opening Scenario File

In [2]:


def parse_scenario_file(file_path: str) -> Dict[Tuple[int, int, int], float]:
    with open(file_path, 'r') as file:
        content = file.read()

    # Extract the C_ijk values
    values = re.findall(r'\d+(?:\.\d+)?', content)
    values = [float(v) for v in values[1:]]  # Skip the first value (number of nodes)

    # Organize the data into a dictionary
    scenario_data = {}
    n_nodes = int(len(values) ** (1/3))  # Cube root of total values gives number of nodes
    for i in range(n_nodes):
        for j in range(n_nodes):
            for k in range(3):  # 3 paths
                index = i * n_nodes * 3 + j * 3 + k
                scenario_data[(i, j, k)] = values[index]

    return scenario_data

def parse_all_scenarios(directory: str) -> Dict[int, Dict[Tuple[int, int, int], float]]:
    all_scenarios = {}
    scenario_files = [f for f in os.listdir(directory) if f.startswith("Scenario") and f.endswith(".dat")]
    
    for file in scenario_files:
        scenario_number = int(re.search(r'Scenario(\d+)', file).group(1))
        file_path = os.path.join(directory, file)
        scenario_data = parse_scenario_file(file_path)
        all_scenarios[scenario_number] = scenario_data

    return all_scenarios

def print_scenario_summary(scenarios: Dict[int, Dict[Tuple[int, int, int], float]]):
    print(f"Total number of scenarios: {len(scenarios)}")
    for scenario_num, scenario_data in scenarios.items():
        n_nodes = int(len(scenario_data) ** (1/3))
        print(f"\nScenario {scenario_num}:")
        print(f"  Number of nodes: {n_nodes}")
        print(f"  Total C_ijk values: {len(scenario_data)}")
        print("  Sample values:")
        for (i, j, k), value in list(scenario_data.items())[:5]:
            print(f"    C_{i},{j},{k} = {value}")

# Example usage
if __name__ == "__main__":
    directory_path = "/Users/vivekchaudhary/Documents/progressive_hedging/MPTSPs/MPTSPs_D0_50"  # Replace with the actual directory path
    all_scenario_data = parse_all_scenarios(directory_path)
    print_scenario_summary(all_scenario_data)

    # You can now work with the parsed data in all_scenario_data
    # For example, to access the value for C_ijk in Scenario 1 where i=0, j=1, k=2:
    # value = all_scenario_data[1][(0, 1, 2)]

Total number of scenarios: 100

Scenario 23:
  Number of nodes: 10
  Total C_ijk values: 1200
  Sample values:
    C_0,0,0 = 0.0
    C_0,0,1 = 0.0
    C_0,0,2 = 255.0
    C_0,1,0 = 726.0
    C_0,1,1 = 96.0

Scenario 37:
  Number of nodes: 10
  Total C_ijk values: 1200
  Sample values:
    C_0,0,0 = 0.0
    C_0,0,1 = 0.0
    C_0,0,2 = 29.0
    C_0,1,0 = 74.0
    C_0,1,1 = 119.0

Scenario 36:
  Number of nodes: 10
  Total C_ijk values: 1200
  Sample values:
    C_0,0,0 = 0.0
    C_0,0,1 = 0.0
    C_0,0,2 = 162.0
    C_0,1,0 = 2054.0
    C_0,1,1 = 94.0

Scenario 22:
  Number of nodes: 10
  Total C_ijk values: 1200
  Sample values:
    C_0,0,0 = 0.0
    C_0,0,1 = 0.0
    C_0,0,2 = 88.0
    C_0,1,0 = 53.0
    C_0,1,1 = 259.0

Scenario 34:
  Number of nodes: 10
  Total C_ijk values: 1200
  Sample values:
    C_0,0,0 = 0.0
    C_0,0,1 = 0.0
    C_0,0,2 = 560.0
    C_0,1,0 = 180.0
    C_0,1,1 = 117.0

Scenario 20:
  Number of nodes: 10
  Total C_ijk values: 1200
  Sample values:
    C_0,0,0 = 